In [51]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import gradio as gr

In [52]:
def fetch_stock_data(ticker, past_days):
    try:
        # If user gives small number, still fetch full available data
        df = yf.download(ticker, period=f"{int(past_days)}d")
        if df.empty or len(df) < 5:
            df = yf.download(ticker, start="2010-01-01")  # fallback full history
        return df[['Close']]
    except Exception:
        return pd.DataFrame()

In [53]:
def prepare_data(scaled_data, time_steps):
    X, y = [], []
    if len(scaled_data) <= time_steps:
        # not enough data: duplicate existing data
        while len(scaled_data) <= time_steps:
            scaled_data = np.concatenate([scaled_data, scaled_data])
    for i in range(time_steps, len(scaled_data)):
        X.append(scaled_data[i-time_steps:i])
        y.append(scaled_data[i])
    return np.array(X), np.array(y)

In [54]:
def build_lstm(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# ✅ CNN Model
def build_cnn(input_shape):
    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=input_shape),
        Flatten(),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [55]:
def predict_stock(ticker, days):
    try:
        # Fetch data dynamically
        data = yf.download(ticker, period=f"{days}d")
        if data.empty:
            return f"⚠️ No data found for '{ticker}'. Try a valid ticker.", None

        # Automatically handle very small datasets
        if len(data) < 2:
          return f"⚠️ Only one data point found for '{ticker}'. Try at least 2 days.", None


        # Prepare data
        df = data[['Close']].copy()
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(df)
        window = max(2, min(5, len(df)//2))  # dynamic window

        X, y = [], []
        for i in range(window, len(scaled_data)):
            X.append(scaled_data[i-window:i, 0])
            y.append(scaled_data[i, 0])
        X, y = np.array(X), np.array(y)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # Simple LSTM model
        model = Sequential([
            LSTM(32, return_sequences=True, input_shape=(X.shape[1], 1)),
            LSTM(32),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X, y, epochs=5, batch_size=4, verbose=0)

        # Predict next day
        last_window = scaled_data[-window:]
        last_window = np.reshape(last_window, (1, window, 1))
        pred_scaled = model.predict(last_window)
        pred = scaler.inverse_transform(pred_scaled)
        predicted_value = float(pred[0][0])
        last_close = float(df['Close'].iloc[-1])
        direction = "📈 Up" if predicted_value > last_close else "📉 Down"

        # Plot Actual vs Predicted
        predicted_all = model.predict(X)
        predicted_prices = scaler.inverse_transform(predicted_all)
        actual_prices = scaler.inverse_transform(y.reshape(-1, 1))

        plt.figure(figsize=(8, 5))
        plt.plot(actual_prices, label='Actual', linewidth=2)
        plt.plot(predicted_prices, label='Predicted', linestyle='--')
        plt.title(f'{ticker} Stock Price Prediction')
        plt.xlabel('Days')
        plt.ylabel('Price ($)')
        plt.legend()
        plt.tight_layout()
        img_path = "prediction_plot.png"
        plt.savefig(img_path)
        plt.close()

        return (
            f"✅ Tomorrow Prediction for **{ticker}**\n"
            f"Predicted Price: **${predicted_value:.2f}**\n"
            f"Current Price: **${last_close:.2f}**\n"
            f"Direction: **{direction}**"
        ), img_path

    except Exception as e:
        return f"⚠️ Error: {str(e)}", None


In [56]:
ui = gr.Interface(
    fn=predict_stock,
    inputs=[
        gr.Textbox(label="Enter Stock Ticker (e.g., AAPL, TSLA, GOOGL)"),
        gr.Number(label="Enter Number of Past Days (e.g., 1, 7, 30, 365, 2000)")
    ],
    outputs=["text", "image"],
    title="📊 Real-Time Stock Price Direction Predictor",
    description="Enter any stock ticker and any number of days to predict if the next day's price will go Up 📈 or Down 📉"
)

ui.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06c12c25b0f4df4d5a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
